# Part 1: Cohort characterization

In this document, we will focus on characterization of the clinical parameters of the cohorts obtained from routine blood tests. We will also visualize the metadata of the participants, such as disease state, age, sex, etc.

## Reading and prep of data

In [ ]:
source("diabetes_analysis_v07.R")

Let's read the file with clinical metadata. 

In [ ]:
md  <- read_xlsx("../data/metadata_v07.xlsx")

In [ ]:
md  %>% colnames

We will group the patients based on age, sex and disease status, which will allow visualization of other metadata in these groups later on.

In [ ]:
options(repr.plot.width = 20)
md  <- md %>% 
mutate(yy = 1)  %>% 
mutate(name = paste(substr(patient,1,1), age_group, sex, Patient_Time))  

In [ ]:
order(levels(factor(md$name)))

In [ ]:
dir.create("../figures/cohort_characterization/basic_metadata/", recursive = T)

## Cohort characterization - descriptive statistics

In  this section, we will create a summary statistics of participants of the study. 

In [ ]:
md  %>% colnames

The groups can be identified from the donor IDs - those starting with 1 are T1D donors, those starting with 2 are Healthy donors. 

In [ ]:
md  <- md  %>% mutate(group = ifelse(substr(patient,1,1)==1,"T1D","Healthy"))

In [ ]:
md  %>% group_by(substr(patient,1,1), time_taken)  %>% 
summarise(mean_age = mean(age_taken, na.rm = TRUE),
          sd_age = sd(age_taken, na.rm = TRUE))

For easy exploratory analysis, we will use the [dlookr package](https://choonghyunryu.github.io/dlookr/articles/EDA.html). 

In [ ]:
library(dlookr)

In [ ]:
md  %>% group_by(group, time_taken)  %>% 
describe() 

In [ ]:
colnames(md)

Let's describe continuous variables:

In [ ]:
table_numeric  <- md  %>% group_by(group, time_taken)  %>% 
describe()  %>% 
dplyr::filter(described_variables %in% c("age_taken", "bmi_man", "hba1c_man", 
                                         "fasting_cpept_1", "gad_man", "iaa_man", "ia2_man", "znt8_man",
                                       "lymph_abs", "mon_abs", "neutr_abs", "eoz_abs", "baz_abs",
                                         "lymph_perc", "mon_perc", "neutr_perc", "eoz_perc", "baz_perc"))  %>% 
dplyr::select(1:7)
table_numeric

In [ ]:
dir.create("../tables/cohort_characterization/")

In [ ]:
write.csv(table_numeric, "../tables/cohort_characterization/table_numeric.csv")

Testing differences between healthy and T1D:

In [ ]:
 md  %>% 
dplyr::select(group, time_taken, age_taken, bmi_man, hba1c_man, fasting_cpept_1,lymph_abs, mon_abs, neutr_abs, eoz_abs, baz_abs,
                                         lymph_perc, mon_perc, neutr_perc, eoz_perc, baz_perc)  %>% 
dplyr::filter(time_taken == "T0")  %>% 
pivot_longer(cols = c(3:16), names_to = "variable", values_to = "value")  %>% 
group_by(variable)  %>% 
rstatix::wilcox_test(value ~ group)

In [ ]:
 md  %>% 
dplyr::select(group, time_taken, age_taken, bmi_man, hba1c_man, fasting_cpept_1,lymph_abs, mon_abs, neutr_abs, eoz_abs, baz_abs,
                                         lymph_perc, mon_perc, neutr_perc, eoz_perc, baz_perc)  %>% 
dplyr::filter(time_taken == "T0")

For categorical variables we will summarize sex and HLA haplotypes. We will use the columns of metadata representing HLA allele typing from raw fastq reads to assemble the HLA-DQ2 HLA-DQ8 genotypes.

In [ ]:
table_category  <- md  %>% group_by(group, time_taken)  %>% 
 mutate(dq2 = ifelse((HLA_DQA11 == "DQA1*05:01:01" | HLA_DQA12 == "DQA1*05:01:01") &
                                   (HLA_DQB11 == "DQB1*02:01:01" | HLA_DQB12 == "DQB1*02:01:01"),"DQ2",
                                   "Other"),
                      dq8 = ifelse((HLA_DQA11 == "DQA1*03:01:01" | HLA_DQA12 == "DQA1*03:01:01") &
                                   (HLA_DQB11 == "DQB1*03:02:01" | HLA_DQB12 == "DQB1*03:02:01"),"DQ8",
                                   "Other"))  %>% 
mutate(dq2_8 = ifelse(dq2 == "DQ2" & dq8 == "DQ8","DQ2_8",
                                     ifelse(dq2 == "DQ2","DQ2", ifelse(dq8 == "DQ8","DQ8","Other"))))  %>% 
diagnose_category(sex, dq2, dq8, dq2_8) 

table_category

In [ ]:
write.csv(table_category, "../tables/cohort_characterization/table_category.csv")

# Fig. 1: Schematic diagram

In this part, we will create a diagram that visualizes the cohort characteristics in Fig 1 of the manuscript.

## Autoantibodies - levels

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 3)

md  %>% filter(time_taken == "T0")  %>% 
ggplot(aes(x = name, y = yy)) + geom_point(aes(size = gad_man, colour = gad_man)) + 
scale_color_gradientn(colors = c("grey50","indianred1", "indianred2", "indianred3")) +
  theme_bw() + theme(axis.text.x = element_blank(),
                    axis.text.y = element_blank(),
                    axis.ticks = element_blank(),
                    panel.grid = element_blank(),
                    ) + ggtitle("GAD") + NoLegend()
ggsave("../figures/cohort_characterization/basic_metadata/gad.svg", width = 23, height = 3, units = "cm")

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 3)

md  %>% filter(time_taken == "T0")  %>% 
ggplot(aes(x = name, y = yy)) + geom_point(aes(size = iaa_man, colour = iaa_man)) + 
scale_color_gradientn(colors = c("grey50","indianred1", "indianred2", "indianred3")) +
   theme_bw() + theme(axis.text.x = element_blank(),
                    axis.text.y = element_blank(),
                    axis.ticks = element_blank(),
                    panel.grid = element_blank(),
                    ) + ggtitle("IAA") + NoLegend()
ggsave("../figures/cohort_characterization/basic_metadata/iaa.svg", width = 23, height = 3, units = "cm")

In [ ]:
md  %>% filter(time_taken == "T0")  %>% 
ggplot(aes(x = name, y = yy)) + geom_point(aes(size = ia2_man, colour = ia2_man)) + 
scale_color_gradientn(colors = c("grey50","indianred1", "indianred2", "indianred3")) +
   theme_bw() + theme(axis.text.x = element_blank(),
                    axis.text.y = element_blank(),
                    axis.ticks = element_blank(),
                    panel.grid = element_blank(),
                    ) + ggtitle("IA2") + NoLegend()
ggsave("../figures/cohort_characterization/basic_metadata/ia2.svg", width = 23, height = 3, units = "cm")

In [ ]:
md  %>% filter(time_taken == "T0")  %>% 
ggplot(aes(x = name, y = yy)) + geom_point(aes(size = znt8_man, colour = znt8_man)) + 
scale_color_gradientn(colors = c("grey50","indianred1", "indianred2", "indianred3")) +
  theme_bw() + theme(axis.text.x = element_blank(),
                    axis.text.y = element_blank(),
                    axis.ticks = element_blank(),
                    panel.grid = element_blank(),
                    ) + ggtitle("ZNT8") + NoLegend()
ggsave("../figures/cohort_characterization/basic_metadata/znt8.svg", width = 23, height = 3, units = "cm")

## Autoantibodies - positive/negative

In [ ]:
md  %>% filter(time_taken == "T0")   %>% dplyr::select(gad_man)  %>% summary()

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 3)

md  %>% filter(time_taken == "T0")  %>% 
ggplot(aes(x = name, y = yy)) + geom_point(aes(size = gad_man>0.6, colour = gad_man>0.6)) + 
scale_color_manual(values = c("grey50","indianred2", "indianred3")) +
scale_size_manual(values = c(1,2.5)) +
  theme_bw() + theme(axis.text.x = element_blank(),
                    axis.text.y = element_blank(),
                    axis.ticks = element_blank(),
                    panel.grid = element_blank(),
                    ) + ggtitle("GAD") + NoLegend()
ggsave("../figures/cohort_characterization/basic_metadata/gad_2.svg", width = 23, height = 3, create.dir = TRUE, units = "cm")

In [ ]:
md  %>% filter(time_taken == "T0")   %>% dplyr::select(iaa_man)  %>% summary()

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 3)

md  %>% filter(time_taken == "T0")  %>% 
ggplot(aes(x = name, y = yy)) + geom_point(aes(size = iaa_man>0.2, colour = iaa_man>0.2)) + 
scale_color_manual(values = c("grey50","indianred2", "indianred3")) +
scale_size_manual(values = c(1,2.5)) +
  theme_bw() + theme(axis.text.x = element_blank(),
                    axis.text.y = element_blank(),
                    axis.ticks = element_blank(),
                    panel.grid = element_blank(),
                    ) + ggtitle("IAA") + NoLegend()
ggsave("../figures/cohort_characterization/basic_metadata/iaa_2.svg", width = 23, height = 3, units = "cm")

In [ ]:
md  %>% filter(time_taken == "T0")   %>% dplyr::select(ia2_man)  %>% summary()

In [ ]:
md  %>% filter(time_taken == "T0")  %>% 
ggplot(aes(x = name, y = yy)) + geom_point(aes(size = ia2_man>0.8, colour = ia2_man>0.8)) + 
scale_color_manual(values = c("grey50","indianred2", "indianred3")) +
scale_size_manual(values = c(1,2.5)) +
   theme_bw() + theme(axis.text.x = element_blank(),
                    axis.text.y = element_blank(),
                    axis.ticks = element_blank(),
                    panel.grid = element_blank(),
                    ) + ggtitle("IA2") + NoLegend()
ggsave("../figures/cohort_characterization/basic_metadata/ia2_2.svg", width = 23, height = 3, units = "cm")

In [ ]:
md  %>% filter(time_taken == "T0")   %>% dplyr::select(znt8_man)  %>% summary()

In [ ]:
md  %>% filter(time_taken == "T0")  %>% 
ggplot(aes(x = name, y = yy)) + geom_point(aes(size = znt8_man>15, colour = znt8_man>15)) + 
scale_color_manual(values = c("grey50","indianred2", "indianred3")) +
scale_size_manual(values = c(1,2.5)) +
  theme_bw() + theme(axis.text.x = element_blank(),
                    axis.text.y = element_blank(),
                    axis.ticks = element_blank(),
                    panel.grid = element_blank(),
                    ) + ggtitle("ZNT8") + NoLegend()
ggsave("../figures/cohort_characterization/basic_metadata/znt8_2.svg", width = 23, height = 3, units = "cm")

## Categorical metadata

In [ ]:
md  %>% filter(time_taken == "T0")  %>% 
ggplot(aes(x = name, y = yy)) + geom_point(aes(colour = as.factor(age_group)), size = 4) + 
scale_color_manual(values = c("#afe9afff","#37c837ff", "#217821ff")) +
  theme_bw() + theme(axis.text.x = element_blank(),
                    axis.text.y = element_blank(),
                    axis.ticks = element_blank(),
                    panel.grid = element_blank(),
                    ) + ggtitle("Age group") + NoLegend()
ggsave("../figures/cohort_characterization/basic_metadata/age.svg", width = 23, height = 3, units = "cm")

In [ ]:
md  %>% filter(time_taken == "T0")  %>% 
ggplot(aes(x = name, y = yy)) + geom_point(aes(colour = as.factor(sex)), size = 4) + 
scale_color_manual(values = c("#ffbcf2ff","#87cefaff")) +
 theme_bw() + theme(axis.text.x = element_blank(),
                    axis.text.y = element_blank(),
                    axis.ticks = element_blank(),
                    panel.grid = element_blank(),
                    ) + ggtitle("Sex") + NoLegend()
ggsave("../figures/cohort_characterization/basic_metadata/sex.svg", width = 23, height = 3, units = "cm")

In [ ]:
md  %>% filter(time_taken == "T0")  %>% 
ggplot(aes(x = name, y = yy)) + geom_point(aes(colour = as.factor(substr(patient,1,1))), size = 4) + 
scale_color_manual(values = c("indianred2","dodgerblue3")) +
  theme_bw() + theme(axis.text.x = element_blank(),
                    axis.text.y = element_blank(),
                    axis.ticks = element_blank(),
                    panel.grid = element_blank(),
                    ) + ggtitle("Disease group") + NoLegend()
ggsave("../figures/cohort_characterization/basic_metadata/disease.svg", width = 23, height = 3, units = "cm")

In [ ]:
colnames(md)

In [ ]:
md  <- md  %>% mutate(dq2 = ifelse((HLA_DQA11 == "DQA1*05:01:01" | HLA_DQA12 == "DQA1*05:01:01") &
                                   (HLA_DQB11 == "DQB1*02:01:01" | HLA_DQB12 == "DQB1*02:01:01"),"DQ2",
                                   "Other"),
                      dq8 = ifelse((HLA_DQA11 == "DQA1*03:01:01" | HLA_DQA12 == "DQA1*03:01:01") &
                                   (HLA_DQB11 == "DQB1*03:02:01" | HLA_DQB12 == "DQB1*03:02:01"),"DQ8",
                                   "Other"))  %>% mutate(
                      dq2_8 = ifelse(dq2 == "DQ2" & dq8 == "DQ8","DQ2_8",
                                     ifelse(dq2 == "DQ2","DQ2", ifelse(dq8 == "DQ8","DQ8","Other"))))
                     

In [ ]:
md  %>% dplyr::select(patient, dq2, dq8, dq2_8)

In [ ]:
md  %>% filter(time_taken == "T0")  %>% 
ggplot(aes(x = name, y = yy)) + geom_point(aes(colour = as.factor(dq2_8)), size = 4) + 
scale_color_manual(values = c("#ff6a6aff","#cd5555ff","#ffaeb9ff","grey50")) +
  theme_bw() + theme(axis.text.x = element_blank(),
                    axis.text.y = element_blank(),
                    axis.ticks = element_blank(),
                    panel.grid = element_blank(),
                    ) + ggtitle("DQ2") + NoLegend()
ggsave("../figures/cohort_characterization/basic_metadata/dq2.svg", width = 23, height = 3, units = "cm")

In [ ]:
md  %>% colnames

In [ ]:
md  %>% filter(time_taken == "T0")  %>% 
ggplot(aes(x = name, y = yy)) + geom_point(aes(colour = as.factor(part_remission_y_n)), size = 4) + 
scale_color_manual(values = c("#ffe680ff","#eed7f4ff","white")) +
  theme_bw() + theme(axis.text.x = element_blank(),
                    axis.text.y = element_blank(),
                    axis.ticks = element_blank(),
                    panel.grid = element_blank(),
                    ) + ggtitle("Remission") 
ggsave("../figures/cohort_characterization/basic_metadata/remission.svg", width = 23, height = 3, units = "cm")

In [ ]:
md  %>% filter(time_taken == "T0")  %>%
mutate(Ketoacidosis = ifelse(ph_man<7.3,"Keto_1","Keto_0"))  %>% 
ggplot(aes(x = name, y = yy)) + geom_point(aes(colour = as.factor(Ketoacidosis)), size = 4) + 
scale_color_manual(values = c("#bee5e1ff","#f7c8baff","white")) +
  theme_bw() + theme(axis.text.x = element_blank(),
                    axis.text.y = element_blank(),
                    axis.ticks = element_blank(),
                    panel.grid = element_blank(),
                    ) + ggtitle("Remission") 
ggsave("../figures/cohort_characterization/basic_metadata/ketoacidosis.svg", width = 23, height = 3, units = "cm")

# Fig. S1: Blood test results

## Clinical metadata

In [ ]:
library(ggbeeswarm)
library(ggpubr)
ggtheme <- function() {
  theme(
    axis.text = element_text(size = 20),
    axis.title = element_text(size = 20),
      plot.title = element_text(size = 20),
    text = element_text(size = 20, colour = "black", family = "Arial"),
    legend.text = element_text(size = 20),
    legend.key.size = unit(10, units = "points")) }

In [ ]:
library(readxl)
md <- read_excel("../data/metadata_v07.xlsx")
df <- as.data.frame(md)
df

In [ ]:
colnames(df)

In [ ]:
library(svglite)

In [ ]:
dir.create("../figures/cohort_characterization/blood_analysis/svg", recursive = T)
dir.create("../figures/cohort_characterization/blood_analysis/png", recursive = T)

In [ ]:
df

In [ ]:
df  %>% colnames

In [ ]:
plot_list <- list()
j = 1

for(i in c(7,21,22:32)){
    
  df_filt <- df %>% mutate(group = if_else(substr(patient,1,1)=="1","Dia","Ctrl")) %>% 
    mutate(group = paste(group, time_taken))
     df_filt <- df_filt[,c(1:10,77,i)]
  colnames(df_filt) <- c(colnames(df_filt[1:10]), "group","value")
  df_filt$value <- as.numeric(df_filt$value)

    plot  <- df_filt  %>% 
 ggplot(aes(x = group, y = value)) + 
 geom_violin(alpha = 0.3, aes(fill = group), scale = "width") + 
stat_summary(fun = "median",
               geom = "crossbar", 
               width = 0.75,
               color = "grey30") +
geom_beeswarm(size = 3, aes(fill = group), cex = 3, 
                shape = 21, color = "black", corral = "random") +
 stat_compare_means(label = "p.format") +
scale_fill_manual(values = c("#1874cdff",  "#ee6363ff", "#c41515ff","#eeb4b4ff")) +
scale_color_manual(values = c("dodgerblue3",  "#e54c4cff", "#aa2a2aff","#e88989ff")) +
ylab("") +
  xlab("") +
  ylim(0,NA) +
  theme_classic() +
theme(strip.background = element_blank(), panel.grid = element_blank()) + 
  theme(axis.text = element_text(color = "black"),
        axis.text.x = element_text(angle = 90),
       axis.line = element_line(color = "black", size = 0.5),
        axis.ticks.x = element_blank()) +
  ggtitle(colnames(df)[i]) + ggtheme()
    
ggsave(paste0("../figures/cohort_characterization/blood_analysis/png/",paste0("plot_",i),".png"), create.dir = TRUE, width = 12, height = 10, units = "cm")
ggsave(paste0("../figures/cohort_characterization/blood_analysis/svg/",paste0("plot_",i),".svg"), create.dir = TRUE, width = 12, height = 10, units = "cm")
    
    plot_list[[j]]  <- plot
    
    j = j+1
}

Paired statistics separately for the two groups

In [ ]:
plot_list <- list()
j = 1

for(i in c(7,21,22:32)){
    
    df_filt <- df %>% mutate(Condition = if_else(substr(patient,1,1)=="1","Dia","Ctrl")) %>% 
    mutate(Condition = paste(Condition, time_taken))  %>% 
    mutate(Condition = ifelse(Condition == "Ctrl T1", "Ctrl T0", Condition))
     df_filt <- df_filt[,c(1:10,77,i)]
  colnames(df_filt) <- c(colnames(df_filt[1:10]), "Condition","value")
  df_filt$value <- as.numeric(df_filt$value)

    plot  <- df_filt  %>% 
 ggplot(aes(x = Condition, y = value)) + 
 geom_violin(alpha = 0.3, aes(fill = Condition), scale = "width") + 
stat_summary(fun = "median",
               geom = "crossbar", 
               width = 0.75,
               color = "grey30") +
geom_beeswarm(size = 3, aes(fill = Condition), cex = 3, 
                shape = 21, color = "black", corral = "random") +
ggpubr::stat_compare_means(label.x = 1.2, label.y.npc = "top", comparisons = list(c(1,2)), 
                           size = 5, vjust = 0.3, label = "p.format") +
    scale_fill_manual(values = c("#1874cdff",  "#ee6363ff", "#c41515ff","#eeb4b4ff")) +
scale_color_manual(values = c("dodgerblue3",  "#e54c4cff", "#aa2a2aff","#e88989ff")) +
ylab("") +
  xlab("") +
  ylim(0,NA) +
  theme_classic() +
theme(strip.background = element_blank(), panel.grid = element_blank()) + 
  theme(axis.text = element_text(color = "black"),
        axis.text.x = element_text(angle = 90),
       axis.line = element_line(color = "black", size = 0.5),
        axis.ticks.x = element_blank()) +
  ggtitle(colnames(df)[i]) + ggtheme() + NoLegend()
    
 ###############   
        df_filt <- df %>% mutate(Condition = if_else(substr(patient,1,1)=="1","Dia","Ctrl")) %>% 
    mutate(Condition = paste(Condition, time_taken))  %>% 
    dplyr::filter(Condition %in% c("Dia T0", "Dia T1"))
     df_filt <- df_filt[,c(1:10,77,i)]
  colnames(df_filt) <- c(colnames(df_filt[1:10]), "Condition","value")
  df_filt$value <- as.numeric(df_filt$value)
    
df_filt <- df_filt  %>% dplyr::select(patient, time_taken, value)  %>% 
pivot_wider(names_from = time_taken, values_from = value) 
colnames(df_filt)  <- c("Patient_ID","Time0","Time1")
 df_filt <- df_filt %>% dplyr::filter(!is.na(Time1)&(!is.na(Time0)))
    df_filt <- df_filt  %>% pivot_longer(!Patient_ID, names_to = "TimePoint", values_to = "value")
    
plot2 <- df_filt  %>% 
  ggplot(aes(x = TimePoint,
             y = value)) +
geom_violin(alpha = 0.3, scale = "width") + 
stat_summary(fun = "median",
               geom = "crossbar", 
               width = 0.75,
               color = "grey30") +
geom_beeswarm(size = 3, cex = 3, 
                shape = 21, color = "black", corral = "random") +
ggpubr::stat_compare_means(label.x = 1.2, label.y.npc = "top", paired = TRUE, 
                           size = 5, vjust = 0.3, label = "p.format") +
scale_fill_manual(values = c("#ee6363ff", "#c41515ff","#eeb4b4ff")) +
scale_color_manual(values = c("#e54c4cff", "#aa2a2aff","#e88989ff")) +
ylab("") +
  xlab("") +
  ylim(0,NA) +
  theme_classic() +
theme(strip.background = element_blank(), panel.grid = element_blank()) + 
  theme(axis.text = element_text(color = "black"),
        axis.text.x = element_text(angle = 90),
       axis.line = element_line(color = "black", size = 0.5),
        axis.ticks.x = element_blank()) +
  ylim(0,NA)+
  plot_annotation(
    title = paste(stringr::str_replace_all(string = colnames(df)[i], pattern = "/", replacement = "\n")) ,
    theme = theme(plot.title = element_text(size = 8), 
                  plot.subtitle = element_text(size = 8),
                 axis.text.x = element_blank())) + ggtheme() + NoLegend()
  
   plot  <- plot + plot2 
    plot
 #################   
    
ggsave(paste0("../figures/cohort_characterization/blood_analysis/png_paired/",paste0("plot_",i),".png"), 
       width = 15, height = 10, units = "cm", create.dir = T)
ggsave(paste0("../figures/cohort_characterization/blood_analysis/svg_paired/",paste0("plot_",i),".svg"), 
       width = 15, height = 10, units = "cm", create.dir = T)
    
    plot_list[[j]]  <- plot
    
    j = j+1
}

## C-peptide correlations

In [ ]:
df  %>% colnames

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
  df %>%  ggplot(aes(x=fasting_cpept_1, y=fasting_cpept_T1)) +
  geom_point(shape = 16, size = 2) +
 geom_smooth(method=lm) + ggtitle("") +stat_cor() + theme_classic() + ggtheme() +
 scale_y_continuous() + 
 scale_x_continuous(expand = c(0.05,0), limits = c(0,750))  


In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
  df %>%  ggplot(aes(x=fasting_cpept_1, y=fasting_cpept_T1)) +
  geom_point(shape = 16, size = 2, aes(color = factor(part_remission_y_n))) +
 geom_smooth(method=lm) + ggtitle("") +stat_cor() + theme_classic() + ggtheme() +
 scale_y_continuous() + NoLegend() +
 scale_x_continuous(expand = c(0.05,0), limits = c(0,750))  

ggsave("../figures/cohort_characterization/blood_analysis/png/corr_fasting_T0_T1.png", 
       width = 10, height = 10, units = "cm")
ggsave("../figures/cohort_characterization/blood_analysis/png/corr_fasting_T0_T1.svg", 
       width = 10, height = 10, units = "cm")

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)

df %>%  ggplot(aes(x=fasting_cpept_1, y=random_cpeptide_man)) +
  geom_point(shape = 16, size = 2) +
 geom_smooth(method=lm) + ggtitle("") +stat_cor() + theme_classic() + ggtheme() +
 scale_y_continuous() + 
 scale_x_continuous(expand = c(0.05,0), limits = c(0,750))  

ggsave("../figures/cohort_characterization/blood_analysis/png/corr_fasting_random.png", 
       width = 10, height = 10, units = "cm")
ggsave("../figures/cohort_characterization/blood_analysis/png/corr_fasting_random.svg", 
       width = 10, height = 10, units = "cm")

# Standardized mean difference dia vs healthy

In this part, we will perform the standardized mean difference (SMD) analysis to show that cohort is balanced in terms of age. We will use the MatchIt package. 

In [ ]:
library(MatchIt)

In [ ]:
md  <- read_xlsx("../data/metadata_v07.xlsx")

In [ ]:
md  %>% colnames

In [ ]:
md

In [ ]:
md_filt  <- md  %>% dplyr::filter(time_taken == "T0")  %>% 
mutate(condition = substr(patient,1,1))  %>% 
dplyr::select(age_taken, sex, condition)

In [ ]:
md_filt$condition2  <- as.factor(md_filt$condition)
md_filt$condition2  <- gsub(md_filt$condition2, pattern = "2", replacement = 0)

In [ ]:
md_filt$condition  <- NULL

In [ ]:
md_filt$condition2  <- as.numeric(md_filt$condition2)

In [ ]:
m.out0 <- matchit(condition2 ~ .,
                  data = md_filt ,
                  method = NULL,
                  distance = "glm")

summary(m.out0)

In [ ]:
plot(summary(m.out0))


In [ ]:
pdf(file = "../figures/cohort_characterization/age_SMD.pdf", width = 6, height = 4)
plot(summary(m.out0))
dev.off()

# SessionInfo

In [ ]:
sessionInfo()